In [ ]:
import pandas as pd
import json
import plotly
import plotly.express as px
import datetime as dt
from  geopy.geocoders import Nominatim
import meteomatics.api as weather
from api_config import *

In [ ]:
geolocator = Nominatim(user_agent="HeatML")
loc = geolocator.geocode("Hong Kong"+','+ "China")
lat = loc.latitude
lng = loc.longitude

curr_dt = dt.datetime.utcnow()
prev_dt = curr_dt - dt.timedelta(days=30)
curr_dt = dt.datetime.utcnow() + dt.timedelta(days=1)
curr_dt = str(curr_dt.isoformat()) + "Z"
prev_dt = str(prev_dt.isoformat()) + "Z"
interval = 6 # hours

In [ ]:
# url=f"https://api.meteomatics.com/{prev_dt}--{curr_dt}:PT{interval}H/t_apparent:F/{lat},{lng}/json?model=mix"
url=f"https://api.meteomatics.com/{prev_dt}--{curr_dt}:PT{interval}H/t_apparent:F,soil_moisture_deficit:mm,forest_fire_warning:idx,co:ugm3,o3:ugm3,no2:ugm3,t_1000m:F,t_2m:F/{lat},{lng}/json?model=mix"

In [ ]:
df = weather.query_api(url=url, username=USER, password=PASSWORD)
response = dict(df.json())
response

In [ ]:
data = response["data"]
apparent, soil, fire, co, o3, no2, t1000, t2 = data
apparent, soil, fire, co, o3, no2, t1000, t2 = apparent["coordinates"][0]["dates"], soil["coordinates"][0]["dates"], fire["coordinates"][0]["dates"], co["coordinates"][0]["dates"], o3["coordinates"][0]["dates"], no2["coordinates"][0]["dates"], t1000["coordinates"][0]["dates"], t2["coordinates"][0]["dates"]

In [ ]:
df1 = pd.DataFrame(apparent).rename(columns={"value": "Heat Index (F)", "date": "Datetime"})
df2 = pd.DataFrame(soil).rename(columns={"value": "Soil Moisture (mm)", "date": "Datetime"})
df3 = pd.DataFrame(fire).rename(columns={"value": "Forest Fire Index", "date": "Datetime"})
df4 = pd.DataFrame(co).rename(columns={"value": "Carbon Monoxide (ugm3)", "date": "Datetime"})
df5 = pd.DataFrame(o3).rename(columns={"value": "Ozone (ugm3)", "date": "Datetime"})
df6 = pd.DataFrame(no2).rename(columns={"value": "Nitrogen Dioxide (ugm3)", "date": "Datetime"})
df7 = pd.DataFrame(t1000).rename(columns={"value": "Temp @ 1km (F)", "date": "Datetime"})
df8 = pd.DataFrame(t2).rename(columns={"value": "Temp @ 2m (F)", "date": "Datetime"})
df = df1
df["Soil Moisture (mm)"] = df2["Soil Moisture (mm)"]
df["Forest Fire Index"] = df3["Forest Fire Index"]
df["Carbon Monoxide (ugm3)"] = df4["Carbon Monoxide (ugm3)"]
df["Ozone (ugm3)"] = df5["Ozone (ugm3)"]
df["Nitrogen Dioxide (ugm3)"] = df6["Nitrogen Dioxide (ugm3)"]
df["Temp Diff 1km vs 2m (F)"] = df7["Temp @ 1km (F)"].subtract(df8["Temp @ 2m (F)"])
df


In [ ]:
fig = px.line(df, x='Datetime', y=["Ozone (ugm3)", "Nitrogen Dioxide (ugm3)", "Temp Diff 1km vs 2m (F)"])
fig

In [ ]:
start = str(dt.datetime.utcnow().isoformat()) + "Z"
range = (df['date'] > start)
df1 = df.loc[range]
df1

In [ ]:
max_hs = df1["value"].max()

In [ ]:
from scipy import stats

df.date = pd.to_datetime(df.date)
df['date_ordinal'] = pd.to_datetime(df['date']).map(dt.datetime.toordinal)
slope, intercept, r_value, p_value, std_err = stats.linregress(df['date_ordinal'], df['value'])
print(slope,
intercept,
r_value,p_value,
std_err)